## Train a U-Net (Test arch, image type)
#### Train a Hamamtsu HCC U-Net from the scratch.
#### Grayscale image.
#### Add a dense layer at the end.

2 classes: binary_crossentropy, sigmoid, mask(0, 255), flag_multi_class=False

multi classes (2 included) : catocaries_crossentropy, softmax, mask(0, 1, 2, ...), flag_multi_class=True (chosen)

In [1]:
from keras.models import Input, Model
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization, Dense, Flatten
from keras.optimizers import Adam, SGD
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
'''
U-Net: Convolutional Networks for Biomedical Image Segmentation
(https://arxiv.org/abs/1505.04597)
---
img_shape: (height, width, channels)
out_ch: number of output channels
start_ch: number of channels of the first conv
depth: zero indexed depth of the U-structure
inc_rate: rate at which the conv channels will increase
activation: activation function after convolutions
dropout: amount of dropout in the contracting part
batchnorm: adds Batch Normalization if true
maxpool: use strided conv instead of maxpooling if false
upconv: use transposed conv instead of upsamping + conv if false
residual: add residual connections around each conv block if true
'''

def conv_block(m, dim, acti, bn, res, do=0):
	n = Conv2D(dim, 3, activation=acti, padding='same')(m)
	n = BatchNormalization()(n) if bn else n
	n = Dropout(do)(n) if do else n
	n = Conv2D(dim, 3, activation=acti, padding='same')(n)
	n = BatchNormalization()(n) if bn else n
	return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
	if depth > 0:
		n = conv_block(m, dim, acti, bn, res)
		m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
		m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
		if up:
			m = UpSampling2D()(m)
			m = Conv2D(dim, 2, activation=acti, padding='same')(m)
		else:
			m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
		n = Concatenate()([n, m])
		m = conv_block(n, dim, acti, bn, res)
	else:
		m = conv_block(m, dim, acti, bn, res, do)
	return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
		 dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
	i = Input(shape=img_shape)
	o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
# 	o = Conv2D(out_ch, 1, activation='sigmoid')(o)
	o = Conv2D(2, 1, activation=activation)(o)
	o = Flatten()(o)
	o = Dense(out_ch, use_bias=False, activation='sigmoid')(o)
	return Model(inputs=i, outputs=o)
#add a dense layer, modify grayscale to rgb

Using TensorFlow backend.


In [2]:
model = UNet((256,256,3))

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 256, 256, 64) 640         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_47[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_48[0][0]                  
__________________________________________________________________________________________________
conv2d_49 

In [23]:
# model.compile(optimizer = SGD(lr = 1e-2, decay=0.001, momentum=0.9), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [24]:
model.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [25]:
# model.compile(optimizer = Adam(lr = 1e-3), loss = 'jaccard_distance', metrics = ['accuracy'])

In [26]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
#             for one pixel in the image, find the class in mask and convert it into one-hot vector
            index = np.where(mask == i)
            index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
#         mask[mask > 0.5] = 1
#         mask[mask <= 0.5] = 0
    return (img,mask)

In [27]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)


In [28]:
# from keras.models import trainGenerator
# from keras.data import trainGenerator

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


data_gen_args = dict(featurewise_std_normalization=True)

train_generator = trainGenerator(batch_size=16, train_path='E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image', image_folder='image_unet_train', mask_folder='mask_unet_train', aug_dict=data_gen_args, target_size=(256,256),  seed = 1, flag_multi_class=False)
validation_generator = trainGenerator(batch_size=16, train_path='E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image', image_folder='image_unet_val', mask_folder='mask_unet_val', aug_dict=data_gen_args, target_size=(256,256),  seed = 1, flag_multi_class=False)

csv_logger = CSVLogger('E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\unet\\dense_lr3_16_training.csv', append=True)
model_checkpoint = ModelCheckpoint('E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\unet\\dense_lr3_16_weights.{epoch:02d}-{val_acc:.4f}.hdf5', monitor='val_loss',verbose=1, save_best_only=True, period=5)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=15, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, min_delta=0.01, cooldown=3, patience=9, min_lr=0.00001, verbose=1)
results = model.fit_generator(train_generator, steps_per_epoch=343,epochs=100, validation_data=validation_generator, validation_steps=68, callbacks=[csv_logger, model_checkpoint, early_stop, reduce_lr])
csv_logger.csv_file.close()


Epoch 1/100
Found 1100 images belonging to 1 classes.
Found 5500 images belonging to 1 classes.
Found 1100 images belonging to 1 classes.
Found 5500 images belonging to 1 classes.


ValueError: Error when checking target: expected dense_2 to have 2 dimensions, but got array with shape (16, 65536, 2)

In [ ]:
model.save('E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\unet\\dense_lr3_16_best_model.hdf5')

In [ ]:
# # =============================================================================
# # Import images
# # =============================================================================
# seed = 42
# random.seed = seed
# np.random.seed = seed

# trainPath=".\\input\\train"
# fileList=[os.path.join(trainPath,fname) for fname in os.listdir(trainPath)]
# fileList=fileList[0:-1] #a cause fichier thumbnail qui fout la merde.

# #load train imgs and masks
# trainList=[]
# trainMask=[]
# for file in fileList:
#     if file.find("tiff")==-1:
#         trainList.append(file)
#     else:
#         trainMask.append(file)
            
# x_train=np.array((len(trainList),64,64,3),dtype='uint8')
# y_train=np.array((len(trainList),64,64,1),dtype='bool')
# x_train = np.array([np.array(cv2.imread(fname)) for fname in trainList])
# #should be n,x,y,c false/true
# y_train = np.array([np.array(cv2.imread(fname,0)).astype('bool') for fname in trainMask])
# y_train= np.reshape(y_train,(77,256,256,1))

# #load validation images and masks
# valPath=".\\input\\val"
# fileList=[os.path.join(valPath,fname) for fname in os.listdir(valPath)]
# fileList=fileList[0:-1] #a cause fichier thumbnail qui fout la merde
# valList=[]
# valMask=[]
# for file in fileList:
#     if file.find("mask")==-1:
#         valList.append(file)
#     else:
#         valMask.append(file)
            
# x_val=np.array((len(valMask),256,256,3),dtype='uint8')
# y_val=np.array((len(valMask),256,256,1),dtype='bool')
# x_val = np.array([np.array(cv2.imread(fname)) for fname in valList])
# y_val = np.array([np.array(cv2.imread(fname,0)).astype('bool') for fname in valMask])
# y_val=np.reshape(y_val,(20,256,256,1))

In [ ]:
# =============================================================================
# Training
# =============================================================================
# callbacks_list = [
#         callbacks.EarlyStopping(
#                 monitor='val_acc',
#                 patience=20,
#                 verbose=1
#                 ),
                
#         callbacks.ModelCheckpoint(
#                 filepath='Unet_baseline.h5',
#                 monitor='val_acc',
#                 save_best_only=True,
#                 verbose=1
#                 ),
                
#         callbacks.ReduceLROnPlateau(
#                 monitor='val_acc',
#                 factor=0.1,
#                 patience=10,
#                 verbose=1
#                 )
        
#                 ]

# results = model.fit(x_train, 
#                     y_train, 
#                     validation_split=0.2, 
#                     batch_size=16, 
#                     epochs=100,
#                     callbacks=callbacks_list)

#plot training results
acc=results.history['acc']
val_acc=results.history['val_acc']
loss=results.history['loss']
val_loss=results.history['val_loss']

epochs=range(1, len(acc)+1)
plt.figure()
plt.xlabel('epochs')
plt.plot(epochs,acc,'b',label='Training acc')
plt.plot(epochs,val_acc,'',label='Validation acc') 
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\unet\\dense_lr3_16_train_val_acc')
plt.figure()
plt.xlabel('epochs')
plt.plot(epochs,loss,'b',label='Training loss')
plt.plot(epochs, val_loss,'',label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\unet\\dense_lr3_16train_val_loss')
plt.show()

In [ ]:
# import cv2
# import os
# def testGenerator(test_path,num_image = 7,target_size = (256,256),flag_multi_class = False):
#     for i in range(num_image):
#         img = cv2.imread(os.path.join(test_path,"%d.tif"%i),0)
        
#         img = img / 255
#         img = cv2.resize(img,target_size)
# #         print(img.shape)
#         img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
#         img = np.reshape(img,(1,)+img.shape)
# #         print(img.shape)
#         yield img

In [ ]:
# nor = [0,0,0]
# tum = [255,255,255]

# COLOR_DICT = np.array([nor, tum])

# def labelVisualize(num_class,color_dict,img):
#     img = img[:,:,0] if len(img.shape) == 3 else img
#     img_out = np.zeros(img.shape + (3,))
#     for i in range(num_class):
#         img_out[img == i,:] = color_dict[i]
#     return img_out


# def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
#     for i,item in enumerate(npyfile):
#         img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
#         img = cv2.resize(img, (64, 64))
#         cv2.imwrite(os.path.join(save_path,"%d_predict.tiff"%i),img*255)

In [ ]:
# testGene = testGenerator("E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image\\test", target_size = (256,256))
# results_test = model.predict_generator(testGene,7,verbose=1)
# # print(sum(results[:,:,:,:]!=0))
# # for i in range(7):
# #     cv2.imwrite("E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image\\test"+'\\'+str(i)+'.tiff',results[i]*255)
# saveResult("E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image\\test",results_test)